# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [3]:
import pandas as pd
import numpy as np

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [238]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df.rename(columns={"Health":"hp_start", "Rate of health decline": "hp_decline_rate"}, inplace=True)
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["bed", 30, 0, True, 5],
          ["ventilator", 10, 0, True, 30],
          ["oxygen mask", 10, 0, True, 20],
          ["plasma", 10, 0, False, 15],
          ["remdesivir", 7, 0, False, 30],
          ["dexamethasone", 20, 0, False, 25],
          ["casirivimab", 10, 0, False, 15],
          ["chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
print(resources_df,"\n")
print(sailors_df)

        resource  count  amt_used reusable  health_bonus
0            bed     30         0     True             5
1     ventilator     10         0     True            30
2    oxygen mask     10         0     True            20
3         plasma     10         0    False            15
4     remdesivir      7         0    False            30
5  dexamethasone     20         0    False            25
6    casirivimab     10         0    False            15
7    chloroquine     17         0    False            10 

    hp_start  hp_decline_rate
ID                           
1        100               -1
2         75               -1
3         65               -7
4         55              -25
5         33              -30
6         89              -50
7         90              -60
8         45              -65
9        100              -79
10        95              -88
11        60              -62
12        72              -18
13        55              -68
14        28              -67
15   

In [249]:
def use_resource(sailors_df, resources_df, resource, indices):
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] = True
    sailors_df.loc[sailors_df[resource] == True, "hp_final"] += bonus_dict[resource]
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] = len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.iloc[:30].index
    use_resource(sailors_df, resources_df, "bed", add_beds)
    
    add_ventilators = sailors_df.iloc[:10].index
    use_resource(sailors_df, resources_df, "ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["ventilator"]!=True) & (sailors_df["bed"]==True)]
    add_oxymask = oxymask_candidates.iloc[:10].index
    use_resource(sailors_df, resources_df, "oxygen mask", add_oxymask)

def ded(sailors_df, resources_df, final_health):
    minarray = []
    for i in range(len(final_health)):
        if final_health.iloc[i] <= 0:
            y = final_health.iloc[i]  
            for j in range(3, len(resources_df)):
                x = y + resources_df.iloc[j,4]
                minarray.append(x)             
            m = min(k for k in minarray if k > 0)
            bonus_used = m - y
            idx = minarray.index(m)
            resource = resources_df["resource"].iloc[idx]
            print(resource)
            use_resource(sailors_df, resources_df, resource, sailors_df.iloc[i:i+1].index)
            minarray = []

def allocation(sailors_df, resources_df):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["hp_final"] = sailors_df["hp_start"]+sailors_df["hp_decline_rate"]
    sailors_df.sort_values(by=["hp_final"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    ded(sailors_df, resources_df, sailors_df["hp_final"])
    
    return sailors_df, resources_df

def calc_final_score(allocation_df):
    final_health = allocation_df["hp_final"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    return final_health.sum()

def generate_output(sailors_df):
    rename_dict = {"hp_start": "Sailor Health",
                   "hp_decline_rate": "Rate of health decline",
                   "hp_final": "Total"}
    return sailors_df.rename(columns=rename_dict)

In [252]:
a_df, r_df = allocation(sailors_df, resources_df)
a_df["remdesivir"] = 0
print(a_df)
generate_output(a_df)
a_df.to_csv("Challenge3_Day1Submission.csv")

ventilator
    hp_start  hp_decline_rate  remdesivir  hp_final   bed ventilator  \
ID                                                                     
20        20              -80           0         5  True       True   
15        39              -89           0        15  True       True   
14        28              -67           0        26  True       True   
8         45              -65           0        45  True       True   
13        55              -68           0        52  True       True   
11        60              -62           0        63  True       True   
5         33              -30           0        68  True       True   
10        95              -88           0        72  True       True   
9        100              -79           0        86  True       True   
4         55              -25           0        95  True       True   
7         90              -60           0        55  True        NaN   
6         89              -50           0        64  